# Part 3: GraphQA using LangChain
In this notebook, we will use the LangChain library to answer questions on top of the Kùzu graph we
just created in the previous section. The example below uses the OpenAI GPT-3.5 turbo model to generate
Cypher and answer questions via a text-to-Cypher pipeline, but you can use any other model and see
how it performs.

We start by opening a connection to the existing database and loading the `OPENAI_API_KEY` variable
from a local `.env` file.

In [14]:
# !uv pip install python-dotenv langchain

In [15]:
import os

import kuzu
from dotenv import load_dotenv

# Load OpenAI API key from .env file
load_dotenv()
assert "OPENAI_API_KEY" in os.environ, "Please set OPENAI_API_KEY in the .env file"

db = kuzu.Database("db/kuzudb")
conn = kuzu.Connection(db)

In [16]:
from langchain.chains import KuzuQAChain
from langchain_community.graphs import KuzuGraph
from langchain_openai import ChatOpenAI

In [17]:
# Create a graph object for KuzuQAChain and print the schema
graph = KuzuGraph(db)
print(graph.get_schema)

Node properties: [{'properties': [('country', 'STRING')], 'label': 'Country'}, {'properties': [('customer_id', 'INT64'), ('name', 'STRING'), ('age', 'INT64')], 'label': 'Customer'}, {'properties': [('id', 'INT64'), ('title', 'STRING'), ('country', 'STRING'), ('description', 'STRING'), ('variety', 'STRING'), ('points', 'INT64'), ('price', 'DOUBLE'), ('state', 'STRING'), ('taster_name', 'STRING'), ('taster_twitter_handle', 'STRING')], 'label': 'Wine'}, {'properties': [('taster_twitter_handle', 'STRING'), ('taster_name', 'STRING'), ('taster_id', 'STRING')], 'label': 'Taster'}]
Relationships properties: [{'properties': [], 'label': 'Tasted'}, {'properties': [], 'label': 'Purchased'}, {'properties': [], 'label': 'LivesIn'}, {'properties': [], 'label': 'Follows'}, {'properties': [], 'label': 'IsFrom'}]
Relationships: ['(:Taster)-[:Tasted]->(:Wine)', '(:Customer)-[:Purchased]->(:Wine)', '(:Customer)-[:LivesIn]->(:Country)', '(:Customer)-[:Follows]->(:Taster)', '(:Wine)-[:IsFrom]->(:Country)']

This schema is passed as part of the prompt to the LLM, which is then used to generate the Cypher query.
The following example shows how to the GPT-3.5 turbo model is used for both text-to-Cypher and for
answer generation.

In [18]:
chain = KuzuQAChain.from_llm(
    llm=ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0, api_key=os.environ.get("OPENAI_API_KEY")),
    graph=graph,
    verbose=True,
)


We can then ask questions in natural language.

In [19]:
chain.invoke("How many wines has Roger Voss tasted?")



> Entering new KuzuQAChain chain...
Generated Cypher:
MATCH (:Taster {taster_name: 'Roger Voss'})-[:Tasted]->(w:Wine)
RETURN COUNT(w)
Full Context:
[{'COUNT(w._ID)': 25514}]

> Finished chain.


{'query': 'How many wines has Roger Voss tasted?',
 'result': 'Roger Voss has tasted 25,514 wines.'}

In [20]:
chain.invoke("Give me the full name of 3 customers who purchased wine that was tasted by Roger Voss?")



> Entering new KuzuQAChain chain...
Generated Cypher:
MATCH (c:Customer)-[:Purchased]->(w:Wine)<-[:Tasted]-(t:Taster)
WHERE t.taster_name = 'Roger Voss'
RETURN c.name AS full_name
LIMIT 3
Full Context:
[{'full_name': 'Christine Wilkinson'}, {'full_name': 'Allison Rodriguez'}, {'full_name': 'Jill Wallace'}]

> Finished chain.


{'query': 'Give me the full name of 3 customers who purchased wine that was tasted by Roger Voss?',
 'result': 'The full names of the 3 customers who purchased wine that was tasted by Roger Voss are Christine Wilkinson, Allison Rodriguez, and Jill Wallace.'}

## Experiment with open source LLMs
You can use different LLMs, including open source ones, for text-to-Cypher and the answer generation stages. See the
[LangChain docs](https://python.langchain.com/v0.2/docs/integrations/graphs/kuzu_db/#use-separate-llms-for-cypher-and-answer-generation)
for such an example.

Open source LLMs can be self-hosted and served on a local endpoint. In
this example, we use a _much_ cheaper locally running `Mistral-7B-OpenOrca-GGUF` model from LMStudio
for text-to-Cypher, followed by OpenAI's GPT-3.5 turbo model for answer generation. We are still able
to call the `ChatOpenAI` class in both cases because LMStudio's local server mimics OpenAI's API endpoints.

Note that cheap and small open source LLMs may not perform as well as the proprietary, general-purpose ones,
so to obtain best performance on Cypher generation (as well as inference from Cypher quuery results),
you may need to fine-tune a more powerful model.

In [21]:
chain = KuzuQAChain.from_llm(
    cypher_llm=ChatOpenAI(base_url="http://localhost:1234/v1", temperature=0, api_key="not_needed"),
    qa_llm=ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0, api_key=os.environ.get("OPENAI_API_KEY")),
    graph=graph,
    verbose=True,
)

In [22]:
chain.invoke("Which country has the most wines with 100 points?")



> Entering new KuzuQAChain chain...
Generated Cypher:
 MATCH (country:Country)<-[:IsFrom]-(wine:Wine) WHERE wine.points = 100 RETURN country, count(*) AS total ORDER BY total DESC LIMIT 1;
Full Context:
[{'country': {'_id': {'offset': 4, 'table': 3}, '_label': 'Country', 'country': 'France'}, 'total': 8}]

> Finished chain.


{'query': 'Which country has the most wines with 100 points?',
 'result': 'France has the most wines with 100 points.'}

## Next steps
Feel free to experiment with other LLMs and see how they perform on your own data. As the natural
language questions become more complex, it might result in incorrect Cypher generation, no matter
how good the underlying LLM. In such cases, a query rewriting step may be required to provide better
context to the cypher-generating LLM.

This notebook is just the starting point of utilizing knowledge graphs for retrieval and QA tasks. You can
look at more advanced pipelines that utilize agents, memory and routers via the LangChain and LlamaIndex frameworks.

Have fun using graphs, and `pip install kuzu`!